# Get Data
* Download sample data from uniclust. Full model should use uniref (but same process). 
* sample for toy experiemnt. Run bert on that data (including setence piece). 

* Future: toy labelled data, check that data already downloaded.. 

* biopython is useful for processing fasta files 


* For final model :
    * uniref 50 : ftp://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref50/uniref50.fasta.gz
    * uniref 90 : ftp://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref90/uniref90.fasta.gz
    * url = http://gwdu111.gwdg.de/~compbiol/uniclust/2018_08/uniclust30_2018_08.tar.gz   -  30% similarity from uniclust. 



* Exampe of building sentence piece : https://github.com/google/sentencepiece/blob/master/python/README.md

* toy BERT (pytorch) :  https://github.com/YuvalPeleg/transformers-workshop

In [4]:
import subprocess
# import keras

import sentencepiece as spm

%matplotlib inline

In [36]:
# !pip install sentencepiece
# ! pip install pytorch_transformers

     |████████████████████████████████| 184kB 5.0MB/s eta 0:00:01
     |████████████████████████████████| 860kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 296kB 14.4MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=4c240979c598821a458282c2f4796f44fb526efca23b54081d9db0959267aa71
  Stored in directory: /Users/danofer/Library/Caches/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [37]:
import torch
## https://github.com/YuvalPeleg/transformers-workshop/blob/master/MLM.ipynb
import pytorch_transformers

####  Code for downloading the raw data
* About 4-6 GB zipped. 
* commented for now - as I manually extracted the files from the zip & directory strucutre

In [5]:
# url = "http://wwwuser.gwdg.de/~compbiol/uniclust/2016_03/uniclust30_2016_03.tar.gz"  ## Older version of above, 30% smaller.

# path = "./data/uniclust"
# subprocess.run(["wget", "-r", "-nc", "-P", path, url])

CompletedProcess(args=['wget', '-r', '-nc', '-P', './data/uniclust', 'http://wwwuser.gwdg.de/~compbiol/uniclust/2016_03/uniclust30_2016_03.tar.gz'], returncode=0)

* I mv the downloaded file from it's subdir to a higher one. 
* I unzip it (from gz and tar). 
* Within it, I access one of the files from the cmd and sample from the command line - getting the first K rows and outputting only their sequences. 
* We could also remove/merge the rare letters down to 3-4  (We have 25 letters in theory, 20 common and 5 very rare ones). https://wiki.thegpm.org/wiki/Amino_acid_symbols 


* We may want to split too long sequences (either in half or overlapping +- multiple times) 
* Our sample sequences is : `uniclust30_2016_1M_sampledSeq.fasta` 

In [2]:
# ! head -323456   ./data/uniclust/uniclust30_2016_03/uniclust30_2016_03_consensus.fasta | grep -v ">" > uniclust30_2016_1M_sampledSeq.fasta

## We know run sentencePiece to construct  a vocab
* output is viewable on the terminal
* https://github.com/google/sentencepiece/blob/master/python/README.md

In [32]:
# spm.SentencePieceTrainer.Train('--input=uniclust30_2016_1M_sampledSeq.fasta --model_prefix=m --vocab_size=500 --input_sentence_size=22345')
spm.SentencePieceTrainer.Train('--input=uniclust30_2016_1M_sampledSeq.fasta --model_prefix=m --vocab_size=1500')

True

In [33]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [34]:
# returns vocab size
print(sp.get_piece_size())

# id <=> piece conversion
print(sp.id_to_piece(2))
print(sp.id_to_piece(29))
print(sp.piece_to_id('KR'))

# returns 0 for unknown tokens (we can change the id for UNK)
print(sp.piece_to_id('__MUST_BE_UNKNOWN__'))

# <unk>, <s>, </s> are defined by default. Their ids are (0, 1, 2)
# <s> and </s> are defined as 'control' symbol.
for i in range(3):
  print(sp.id_to_piece(i), sp.is_control(i))

60
</s>
LA
0
0
<unk> False
<s> True
</s> True


In [35]:
eg_seq = "MRYTVLIALQGALLLLLLIDDGQGQSPYPYPGMPCNSSRQCGLGTCVHSRCAHCSSDGTLCSPEDPTMVWPCCPESSCQLVVGLPSLVNHYNCLPNQCTDSSQCPGGFGCMTRRSKCELCKADGEACNSPYLDWRKDKECCSGYCHTEARGLEGVCIDPKKIFCTPKNPWQLAPYPPSYHQPTTLRPPTSLYDSWLMSGFLVKSTTAPSTQEEEDDY"
print(sp.encode_as_pieces(eg_seq))
print(sp.encode_as_ids(eg_seq))

['▁M', 'R', 'Y', 'T', 'VL', 'I', 'AL', 'Q', 'G', 'A', 'LL', 'LL', 'LL', 'I', 'D', 'D', 'G', 'Q', 'G', 'Q', 'S', 'P', 'Y', 'P', 'Y', 'P', 'G', 'M', 'P', 'C', 'N', 'S', 'S', 'R', 'Q', 'C', 'G', 'LG', 'T', 'C', 'V', 'H', 'S', 'R', 'C', 'A', 'H', 'C', 'S', 'S', 'D', 'G', 'T', 'L', 'C', 'S', 'P', 'E', 'D', 'P', 'T', 'M', 'V', 'W', 'P', 'C', 'C', 'P', 'E', 'S', 'S', 'C', 'Q', 'LV', 'V', 'GL', 'P', 'S', 'LV', 'N', 'H', 'Y', 'N', 'C', 'L', 'P', 'N', 'Q', 'C', 'T', 'D', 'S', 'S', 'Q', 'C', 'P', 'GG', 'F', 'G', 'C', 'M', 'T', 'R', 'R', 'S', 'K', 'C', 'EL', 'C', 'K', 'A', 'D', 'G', 'E', 'A', 'C', 'N', 'S', 'P', 'Y', 'L', 'D', 'W', 'R', 'K', 'D', 'K', 'E', 'C', 'C', 'S', 'G', 'Y', 'C', 'H', 'T', 'E', 'A', 'R', 'G', 'LE', 'G', 'V', 'C', 'I', 'D', 'P', 'K', 'K', 'I', 'F', 'C', 'T', 'P', 'K', 'N', 'P', 'W', 'Q', 'LA', 'P', 'Y', 'P', 'P', 'S', 'Y', 'H', 'Q', 'P', 'T', 'T', 'L', 'R', 'P', 'P', 'T', 'S', 'L', 'Y', 'D', 'S', 'W', 'L', 'M', 'S', 'G', 'F', 'LV', 'K', 'S', 'T', 'T', 'A', 'P', 'S', 'T', 'Q',

### Build language model
* For proteins - we can legitimately look from both directions (no need to mask!) 
* Need to tokenize using sentence piece. 

* Evaluate model.. once it's done, can try to do on more data or/and fine tuning on supervised task / classification